In [73]:
import pandas as pd
import numpy as np
import cv2
import plotly.graph_objects as go
import matplotlib.pyplot as plt

from Libraries.get_absolute_path import get_absolute_path

In [2]:
# Lecture du fichier des métadonnées
df = pd.read_csv(r'CSV\metadatas_with_url.csv')

In [3]:
def mean_pixel_density(img : cv2.imread) -> int:
    """Fonction qui retourne la densité de pixel moyenne de l'image en niveau de gris"""
    return np.mean(img)

In [4]:
img = cv2.imread(get_absolute_path(df.loc[0, "IMG_URL"]), cv2.IMREAD_GRAYSCALE) # Chargement d'une image par cv2

In [5]:
# Création d'un nouveau DataFrame avec pour chaque images deux valeurs qui sont initialisées à 0 : la densité de pixel moyenne de l'image d'origine
# et celle où le masque de contours du poumon a été appliqué.
mean_density = pd.DataFrame({'FILE NAME' : df['FILE NAME'], 'MeanD_FULL_IMAGE' : np.zeros(len(df)), 'MeanD_LUNG_PART' : np.zeros(len(df))})

In [6]:
mean_density.head() # Vérification de la structure

,FILE NAME,MeanD_FULL_IMAGE,MeanD_LUNG_PART
0,COVID-1,0.0,0.0
1,COVID-2,0.0,0.0
2,COVID-3,0.0,0.0
3,COVID-4,0.0,0.0
4,COVID-5,0.0,0.0


In [ ]:
mean_density['LABEL'] = df['LABEL'] # Ajout de la colonne des labels au dataframe mean_density

In [7]:
# Calcul des densités moyennes sur les images complètes
for file_name, img_url in zip(df['FILE NAME'], df['IMG_URL']): # Pour chaque image du dataset
    img = cv2.imread(get_absolute_path(img_url), cv2.IMREAD_GRAYSCALE) # Création de la matrice représentant l'image en niveau de gris
    # Calcul de la densité de pixel moyen et stockage de celle-ci dans la ligne correspondant du dataframe mean_density
    mean_density.loc[mean_density['FILE NAME'] == file_name, 'MeanD_FULL_IMAGE'] = mean_pixel_density(img) 

In [11]:
# Calcul des densités de pixels sur les images masquées
for file_name, img_url, mask_url in zip(df['FILE NAME'], df['IMG_URL'], df['MASK_RESIZED_URL']): # Pour chaque image du dataset
    img = cv2.imread(get_absolute_path(img_url), cv2.IMREAD_GRAYSCALE) # Création de la matrice représentant l'image en niveau de gris
    mask = cv2.imread(get_absolute_path(mask_url), cv2.IMREAD_GRAYSCALE) # Création de la matrice représentant le masque en niveau de gris
    img_masked = cv2.bitwise_and(img, mask) # Application du masque à l'image
    img_masked_bool = img_masked > 0 # Duplication du masque avec False sur les pixels 0 (noirs) et True sur les autres pixels, permettant de faire le calcul de moyenne uniquement sur la zone des poumons
    # Calcul de la densité de pixel moyen et stockage de celle-ci dans la ligne correspondant du dataframe mean_density
    mean_density.loc[mean_density['FILE NAME'] == file_name, 'MeanD_LUNG_PART'] = np.mean(img_masked, where=img_masked_bool)

In [139]:
fig = go.Figure()
fig.add_trace(go.Box(x=mean_density['LABEL'], y=mean_density['MeanD_FULL_IMAGE'], name='full image', offsetgroup=1))
fig.add_trace(go.Box(x=mean_density['LABEL'], y=mean_density['MeanD_LUNG_PART'], name='masked image', offsetgroup=2))
fig.update_layout(title="Mean pixel density distribution", yaxis_title="Mean pixel density", boxmode="group")
fig.show()

In [146]:
outliers = []

range_COVID = [49.15, 175.82]
range_NORMAL = [19.54, 171.60]
range_VIRAL_PNEUMONIA = [58.20, 172.84]
range_LUNG_OPACITY = [39.30, 190.10]

# Identification des outliers
for label, range in zip(mean_density['LABEL'].unique(), [range_COVID, range_NORMAL, range_VIRAL_PNEUMONIA, range_LUNG_OPACITY]):
    for file_name, mean_dens in zip(mean_density.loc[mean_density['LABEL']==label, 'FILE NAME'], mean_density.loc[mean_density['LABEL']==label, 'MeanD_LUNG_PART']):
        if (mean_dens < range[0]) | (mean_dens > range[1]):
            outliers.append(file_name)

print(outliers)

['COVID-121', 'COVID-195', 'COVID-523', 'COVID-697', 'COVID-748', 'COVID-1046', 'COVID-1049', 'COVID-1063', 'COVID-1086', 'COVID-1120', 'COVID-1179', 'COVID-1232', 'COVID-1322', 'COVID-1334', 'COVID-1468', 'COVID-1757', 'COVID-1820', 'COVID-1876', 'COVID-2015', 'COVID-2107', 'COVID-2121', 'COVID-2182', 'COVID-2324', 'COVID-2413', 'COVID-2465', 'COVID-2630', 'COVID-2672', 'COVID-2837', 'COVID-3071', 'COVID-3156', 'COVID-3158', 'COVID-3259', 'COVID-3342', 'NORMAL-4308', 'NORMAL-8975', 'Viral Pneumonia-31', 'Viral Pneumonia-35', 'Viral Pneumonia-155', 'Viral Pneumonia-159', 'Viral Pneumonia-265', 'Viral Pneumonia-310', 'Viral Pneumonia-373', 'Viral Pneumonia-388', 'Viral Pneumonia-653', 'Viral Pneumonia-696', 'Viral Pneumonia-768', 'Viral Pneumonia-796', 'Viral Pneumonia-1010', 'Viral Pneumonia-1051', 'Viral Pneumonia-1118', 'Viral Pneumonia-1259', 'Viral Pneumonia-1281', 'Viral Pneumonia-1316', 'Lung_Opacity-51', 'Lung_Opacity-381', 'Lung_Opacity-489', 'Lung_Opacity-625', 'Lung_Opacity-6

In [147]:
len(outliers) # 68 outliers


68

In [14]:
mean_density.head()

,FILE NAME,MeanD_FULL_IMAGE,MeanD_LUNG_PART,LABEL
0,COVID-1,145.896847,126.684009,COVID
1,COVID-2,150.640530,104.494054,COVID
2,COVID-3,140.598617,101.497864,COVID
3,COVID-4,116.714679,98.898259,COVID
4,COVID-5,167.835796,124.493169,COVID


In [16]:
# Enregistrement du fichier des densités de pixels
mean_density.to_csv(r'..\..\data\processed\mean_density.csv', sep=',', encoding='utf-8', index=False, header=True) # exportation du fichier de densité sous le nom mean_density.csv

# **Calcul des histogrammes des densités de pixels par catégories**

## Etape 1 - Sur les images non masquées

In [143]:
# Importation des fichiers
mean_density = pd.read_csv("CSV/mean_density.csv")
df = pd.read_csv('CSV/metadatas_with_url.csv')

In [97]:
histograms_full_images = { # Initialisation de listes de 256 valeurs qui représenterons au final le pourcentage de pixel ayant la densité i (i index)
    "hist_COVID" : np.zeros(256, dtype=np.float64),
    "hist_NORMAL" : np.zeros(256, dtype=np.float64),
    "hist_VIRAL_PNEUMONIA" : np.zeros(256, dtype=np.float64),
    "hist_LUNG_OPACITY" : np.zeros(256, dtype=np.float64)
}
for label, hist in zip(df["LABEL"].unique(), histograms_full_images.values()): # Pour les quatres pathologies
    for img_url in df.loc[df["LABEL"] == label, "IMG_URL"]: # Pour chaque image de la catégorie actuelle
        img = cv2.imread(get_absolute_path(img_url), cv2.IMREAD_GRAYSCALE) # Lecture de l'image
        hist += cv2.calcHist([img], [0], None, [256], [0, 256]).astype(int).flatten() # Ajout à la liste correspondante le nombre de pixels pour chaque densité
    hist /= hist.sum() # Calcul de la proportion des densités de pixel

### Distribution des densité de pixel entre 1 et 255 (sans les pixels noirs, beaucoup plus fréquents)

In [114]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(255),y=histograms_full_images['hist_COVID'][1:], name="COVID", line_shape='linear'))
fig.add_trace(go.Scatter(x=np.arange(255),y=histograms_full_images['hist_NORMAL'][1:], name="NORMAL", line_shape='linear'))
fig.add_trace(go.Scatter(x=np.arange(255),y=histograms_full_images['hist_VIRAL_PNEUMONIA'][1:], name="VIRAL PNEUMONIA", line_shape='linear'))
fig.add_trace(go.Scatter(x=np.arange(255),y=histograms_full_images['hist_LUNG_OPACITY'][1:], name="LUNG OPACITY", line_shape='linear'))
fig.update_layout(title = 'Distribution des densité de pixel par pathologie sur image non masquée', legend_title = 'legend', xaxis_title='Densité de pixel', yaxis_title='Proportion de pixel')

### Proportion de pixel noir par catégorie

In [124]:
fig = go.Figure()
fig.add_trace(go.Bar(x=['VIRAL_PNEUMONIA', "NORMAL", "COVID", "LUNG OPACITY"], y=[round(histograms_full_images['hist_VIRAL_PNEUMONIA'][0],3),
                                                                                  round(histograms_full_images['hist_NORMAL'][0],3),
                                                                                  round(histograms_full_images['hist_COVID'][0],3),
                                                                                  round(histograms_full_images['hist_LUNG_OPACITY'][0],3)],
                                                                                marker_color=['#00CC96', '#EF553B', '#636EFA', '#AB63FA'], 
                                                                                text=[round(histograms_full_images['hist_VIRAL_PNEUMONIA'][0],3),
                                                                                      round(histograms_full_images['hist_NORMAL'][0],3),
                                                                                      round(histograms_full_images['hist_COVID'][0],3),
                                                                                      round(histograms_full_images['hist_LUNG_OPACITY'][0],3)]))
fig.update_layout(title = 'Proportion de pixel noir par pathologie', yaxis_title='Proportion de pixel')
fig.show()

# Etape 2 - Sur images masquées

In [109]:
histograms_masked_images = {
    "hist_COVID" : np.zeros(256, dtype=np.float64),
    "hist_NORMAL" : np.zeros(256, dtype=np.float64),
    "hist_VIRAL_PNEUMONIA" : np.zeros(256, dtype=np.float64),
    "hist_LUNG_OPACITY" : np.zeros(256, dtype=np.float64)
}
for label, hist in zip(df["LABEL"].unique(), histograms_masked_images.values()):
    for img_url, mask_url in zip(df.loc[df["LABEL"] == label, "IMG_URL"], df.loc[df["LABEL"] == label, "MASK_RESIZED_URL"]):
        img = cv2.imread(get_absolute_path(img_url), cv2.IMREAD_GRAYSCALE)
        mask = cv2.imread(get_absolute_path(mask_url), cv2.IMREAD_GRAYSCALE)
        img_masked = cv2.bitwise_and(img, mask) # Application du masque à l'image
        img_masked_bool = np.where(img_masked > 0, 1, 0).astype('uint8') # Duplication du masque avec False sur les pixels 0 (noirs) et True sur les autres pixels, permettant de faire le calcul de moyenne uniquement sur la zone des poumons
        # Calcul de la densité de pixel moyen et stockage de celle-ci dans la ligne correspondant du dataframe mean_density
        hist += cv2.calcHist([img_masked], [0], img_masked_bool, [256], [0, 256]).astype(int).flatten()
    hist /= hist.sum()

In [113]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(255),y=histograms_masked_images['hist_COVID'][1:], name="COVID", line_shape='linear'))
fig.add_trace(go.Scatter(x=np.arange(255),y=histograms_masked_images['hist_NORMAL'][1:], name="NORMAL", line_shape='linear'))
fig.add_trace(go.Scatter(x=np.arange(255),y=histograms_masked_images['hist_VIRAL_PNEUMONIA'][1:], name="VIRAL PNEUMONIA", line_shape='linear'))
fig.add_trace(go.Scatter(x=np.arange(255),y=histograms_masked_images['hist_LUNG_OPACITY'][1:], name="LUNG OPACITY", line_shape='linear'))
fig.update_layout(title = 'Distribution des densité de pixel par pathologie sur image masquée', legend_title = 'legend', xaxis_title='Densité de pixel', yaxis_title='Proportion de pixel')